<a href="https://colab.research.google.com/github/MARTIN0399/hacking-civico/blob/master/notebooks/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COVID-19 en México: Análisis exploratorio de Datos**

***1)Carga de Datos ***

In [4]:
import io
import statistics
import pandas as pd
import numpy as np 

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
path = ('/content/drive/My Drive/Hacking Cívico/datos_abiertos_covid19.zip')
data = pd.read_csv(path,encoding='Latin-1')

In [7]:
data.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-09-12,114cd3,2,3,27,1,27,27,4,1,2020-07-06,2020-06-29,9999-99-99,97,2,52,1,2,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
1,2020-09-12,0bafd8,2,4,15,2,15,15,106,2,2020-04-29,2020-04-28,2020-05-09,1,2,57,1,97,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,2
2,2020-09-12,18747a,2,4,25,1,25,25,12,1,2020-04-14,2020-04-10,9999-99-99,97,2,44,1,2,2,1,2,2,2,1,2,2,1,2,2,99,1,99,MÃ©xico,99,97
3,2020-09-12,174174,2,4,9,2,12,9,7,1,2020-04-18,2020-04-11,9999-99-99,97,1,28,1,97,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,97
4,2020-09-12,0eae86,2,4,30,2,15,30,4,2,2020-04-21,2020-04-21,9999-99-99,1,2,53,1,97,2,2,2,2,2,2,2,2,2,2,2,2,1,99,MÃ©xico,99,1


In [8]:
path_dic='/content/drive/My Drive/Hacking Cívico/diccionario_datos_covid19/diccionario_datos_covid19/Descriptores_0419.xlsx'
diccionario = pd.read_excel(path_dic) 

In [9]:
diccionario 

,Nº,NOMBRE DE VARIABLE,DESCRIPCIÓN DE VARIABLE,FORMATO O FUENTE
0,1,FECHA_ACTUALIZACION,"La base de datos se alimenta diariamente, esta...",AAAA-MM-DD
1,2,ID_REGISTRO,Número identificador del caso,TEXTO
2,3,ORIGEN,La vigilancia centinela se realiza a través de...,CATÁLOGO: ORIGEN ...
3,4,SECTOR,Identifica el tipo de institución del Sistema ...,CATÁLOGO: SECTOR ...
4,5,ENTIDAD_UM,Identifica la entidad donde se ubica la unidad...,CATALÓGO: ENTIDADES
5,6,SEXO,Identifica al sexo del paciente.,CATÁLOGO: SEXO
6,7,ENTIDAD_NAC,Identifica la entidad de nacimiento del paciente.,CATALÓGO: ENTIDADES
7,8,ENTIDAD_RES,Identifica la entidad de residencia del paciente.,CATALÓGO: ENTIDADES
8,9,MUNICIPIO_RES,Identifica el municipio de residencia del paci...,CATALÓGO: MUNICIPIOS
9,10,TIPO_PACIENTE,Identifica el tipo de atención que recibió el ...,CATÁLOGO: TIPO_PACIENTE


In [10]:
path_cat = '/content/drive/My Drive/Hacking Cívico/diccionario_datos_covid19/diccionario_datos_covid19/Catalogos_0412.xlsx'
sheets = ['ORIGEN', 'SECTOR', 'SEXO', 'TIPO_PACIENTE', 'SI_NO', 'NACIONALIDAD', 'RESULTADO', 'de ENTIDADES', 'MUNICIPIOS']
catalogues = {}
for sheet in sheets:
  catalogue = pd.read_excel(path_cat, sheet_name='Catálogo ' + sheet)
  catalogue.columns = catalogue.columns.str.replace(' ', '_')
  catalogue.columns = catalogue.columns.str.replace('.', ' ')
  catalogue.columns = catalogue.columns.str.lower()
  new_catalogue = {sheet: catalogue}
  catalogues.update(new_catalogue)


In [45]:
catalogues ['NACIONALIDAD']

,clave,descripción
0,1,MEXICANA
1,2,EXTRANJERA
2,99,NO ESPECIFICADO


In [11]:
catalogues ['SEXO']

,clave,descripción
0,1,MUJER
1,2,HOMBRE
2,99,NO ESPECIFICADO


In [12]:
catalogues ['MUNICIPIOS']

,clave_municipio,municipio,clave_entidad
0,1,AGUASCALIENTES,1
1,2,ASIENTOS,1
2,3,CALVILLO,1
3,4,COSÍO,1
4,5,JESÚS MARÍA,1
...,...,...,...
2496,999,NO ESPECIFICADO,32
2497,999,NO ESPECIFICADO,36
2498,997,NO APLICA,97
2499,998,SE IGNORA,98


__Perfilamiento__





In [13]:
# dimensión de los datos
print('Número de filas: {}'.format(data.shape[0]))
print('Número de columnas: {}'.format(data.shape[1]))

Número de filas: 1510311
Número de columnas: 35


In [14]:
# Columnas (variables)
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [16]:
# limpieza del nombre de las columnas, remover espacios, carácteres especiales y pasar a minúsculas
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('.', '')
data.columns = data.columns.str.lower()

In [17]:
# calcular número de valores nulos por columna
for col in data.columns:
  print('Valores nulos en "{}": {}'.format(col,data[col].isna().sum()))

Valores nulos en "fecha_actualizacion": 0
Valores nulos en "id_registro": 0
Valores nulos en "origen": 0
Valores nulos en "sector": 0
Valores nulos en "entidad_um": 0
Valores nulos en "sexo": 0
Valores nulos en "entidad_nac": 0
Valores nulos en "entidad_res": 0
Valores nulos en "municipio_res": 0
Valores nulos en "tipo_paciente": 0
Valores nulos en "fecha_ingreso": 0
Valores nulos en "fecha_sintomas": 0
Valores nulos en "fecha_def": 0
Valores nulos en "intubado": 0
Valores nulos en "neumonia": 0
Valores nulos en "edad": 0
Valores nulos en "nacionalidad": 0
Valores nulos en "embarazo": 0
Valores nulos en "habla_lengua_indig": 0
Valores nulos en "diabetes": 0
Valores nulos en "epoc": 0
Valores nulos en "asma": 0
Valores nulos en "inmusupr": 0
Valores nulos en "hipertension": 0
Valores nulos en "otra_com": 0
Valores nulos en "cardiovascular": 0
Valores nulos en "obesidad": 0
Valores nulos en "renal_cronica": 0
Valores nulos en "tabaquismo": 0
Valores nulos en "otro_caso": 0
Valores nulos 

In [18]:
# tipos de datos
data.dtypes

fecha_actualizacion    object
id_registro            object
origen                  int64
sector                  int64
entidad_um              int64
sexo                    int64
entidad_nac             int64
entidad_res             int64
municipio_res           int64
tipo_paciente           int64
fecha_ingreso          object
fecha_sintomas         object
fecha_def              object
intubado                int64
neumonia                int64
edad                    int64
nacionalidad            int64
embarazo                int64
habla_lengua_indig      int64
diabetes                int64
epoc                    int64
asma                    int64
inmusupr                int64
hipertension            int64
otra_com                int64
cardiovascular          int64
obesidad                int64
renal_cronica           int64
tabaquismo              int64
otro_caso               int64
resultado               int64
migrante                int64
pais_nacionalidad      object
pais_orige

__Limpieza__

In [19]:
# convertimos a fecha algunas columnas
def cast_datetime_cols(df):
    timestamp_cols = [col for col in df.columns if "fecha" in col]
    df[timestamp_cols] = df[timestamp_cols].apply(lambda date_col: pd.to_datetime(date_col, errors="coerce"), axis=0)
    return df

data = cast_datetime_cols(data)

In [20]:
data.head()

,fecha_actualizacion,id_registro,origen,sector,entidad_um,sexo,entidad_nac,entidad_res,municipio_res,tipo_paciente,fecha_ingreso,fecha_sintomas,fecha_def,intubado,neumonia,edad,nacionalidad,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,resultado,migrante,pais_nacionalidad,pais_origen,uci
0,2020-09-12,114cd3,2,3,27,1,27,27,4,1,2020-07-06,2020-06-29,NaT,97,2,52,1,2,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
1,2020-09-12,0bafd8,2,4,15,2,15,15,106,2,2020-04-29,2020-04-28,2020-05-09,1,2,57,1,97,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,2
2,2020-09-12,18747a,2,4,25,1,25,25,12,1,2020-04-14,2020-04-10,NaT,97,2,44,1,2,2,1,2,2,2,1,2,2,1,2,2,99,1,99,MÃ©xico,99,97
3,2020-09-12,174174,2,4,9,2,12,9,7,1,2020-04-18,2020-04-11,NaT,97,1,28,1,97,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,97
4,2020-09-12,0eae86,2,4,30,2,15,30,4,2,2020-04-21,2020-04-21,NaT,1,2,53,1,97,2,2,2,2,2,2,2,2,2,2,2,2,1,99,MÃ©xico,99,1


In [58]:
# unimos los datos con los catalogos para tener mejor entendimiento de ellos
data = (
    # unimos con el catálogo MUNICIPIOS y ENTIDADES
    data.merge(catalogues['de ENTIDADES'], how='left', left_on='entidad_res', right_on='clave_entidad').
    drop(columns=['entidad_res', 'abreviatura']).
    rename(columns={'entidad_federativa':'entidad_res'}).
    merge(catalogues['MUNICIPIOS'], how='left', left_on=['clave_entidad','municipio_res'], right_on=['clave_entidad','clave_municipio']).
    drop(columns=['municipio_res', 'clave_entidad', 'clave_municipio']).
    rename(columns={'municipio':'municipio_res'})
)

In [21]:
# unimos con el catálogo de ENTIDADES
entidades_cols = [col for col in data.columns if "entidad" in col and col != 'entidad_res']
for col in entidades_cols:
    data = (
        data.merge(catalogues['de ENTIDADES'], how='left', left_on=col, right_on='clave_entidad').
        drop(columns=[col, 'abreviatura', 'clave_entidad']).
        rename(columns={'entidad_federativa':col})
    )

In [22]:
# unimos con el catálogo SI_NO
sino_cols = ['intubado', 'neumonia', 'embarazo', 'habla_lengua_indig', 'diabetes', 'epoc', 'asma', 'inmusupr',
             'hipertension', 'otra_com', 'cardiovascular', 'obesidad', 'renal_cronica', 'tabaquismo', 
             'otro_caso', 'migrante', 'uci']
for col in sino_cols:
    data = (
        data.merge(catalogues['SI_NO'], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [23]:
# unimos otras columnnas que comparten lógica
other_cols = ['origen', 'sector', 'sexo', 'tipo_paciente', 'nacionalidad', 'resultado']
for col, cat in zip(other_cols, [x.upper() for x in other_cols]):
    data = (
        data.merge(catalogues[cat], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

KeyError: ignored

In [24]:
# creamos algunas variables que podrían ser útiles después
data['defuncion'] = ~data.fecha_def.isna()
data['mes_def'] = data.fecha_def.dt.month
data['año_def'] = data.fecha_def.dt.year
data['mes_ingreso'] = data.fecha_ingreso.dt.month
data['año_ingreso'] = data.fecha_ingreso.dt.year

In [25]:
data.head()

,fecha_actualizacion,id_registro,entidad_res,municipio_res,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-12,114cd3,27,4,2020-07-06,2020-06-29,NaT,52,1,MÃ©xico,99,TABASCO,TABASCO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,7,2020
1,2020-09-12,0bafd8,15,106,2020-04-29,2020-04-28,2020-05-09,57,1,MÃ©xico,99,MÉXICO,MÉXICO,SI,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,True,5.0,2020.0,4,2020
2,2020-09-12,18747a,25,12,2020-04-14,2020-04-10,NaT,44,1,MÃ©xico,99,SINALOA,SINALOA,NO APLICA,NO,NO,NO,SI,NO,NO,NO,SI,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020
3,2020-09-12,174174,9,7,2020-04-18,2020-04-11,NaT,28,1,MÃ©xico,99,CIUDAD DE MÉXICO,GUERRERO,NO APLICA,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020
4,2020-09-12,0eae86,30,4,2020-04-21,2020-04-21,NaT,53,1,MÃ©xico,99,VERACRUZ DE IGNACIO DE LA LLAVE,MÉXICO,SI,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,SI,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,False,NaN,NaN,4,2020


__Filtros__

Distintas formas de filtrar lo mismo

In [26]:
# usando query 
data.query('nacionalidad == "MEXICANA"').head(3)

,fecha_actualizacion,id_registro,entidad_res,municipio_res,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-12,114cd3,27,4,2020-07-06,2020-06-29,NaT,52,1,MÃ©xico,99,TABASCO,TABASCO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,7,2020
1,2020-09-12,0bafd8,15,106,2020-04-29,2020-04-28,2020-05-09,57,1,MÃ©xico,99,MÉXICO,MÉXICO,SI,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,True,5.0,2020.0,4,2020
2,2020-09-12,18747a,25,12,2020-04-14,2020-04-10,NaT,44,1,MÃ©xico,99,SINALOA,SINALOA,NO APLICA,NO,NO,NO,SI,NO,NO,NO,SI,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020


In [69]:
#haciendo referencia a la columna como ìndice
data[data['nacionalidad'] == 'MEXICANA'].head(3)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-12,114cd3,2020-07-06,2020-06-29,NaT,52,1,MÃ©xico,99,TABASCO,CENTRO,TABASCO,TABASCO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,7,2020
1,2020-09-12,0bafd8,2020-04-29,2020-04-28,2020-05-09,57,1,MÃ©xico,99,MÉXICO,TOLUCA,MÉXICO,MÉXICO,SI,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,True,5.0,2020.0,4,2020
2,2020-09-12,18747a,2020-04-14,2020-04-10,NaT,44,1,MÃ©xico,99,SINALOA,MAZATLÁN,SINALOA,SINALOA,NO APLICA,NO,NO,NO,SI,NO,NO,NO,SI,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,4,2020


In [70]:
# seleccionando directamente la columna que queremos filtrar
data[data.nacionalidad == 'EXTRANJERA'].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
653,2020-09-12,1a500b,2020-05-01,2020-04-24,2020-05-02,86,1,Estados Unidos de AmÃ©rica,99,MORELOS,JOJUTLA,MORELOS,NO ESPECIFICADO,NO,SI,NO APLICA,NO,NO,NO,NO,NO,SI,NO,NO,SI,SI,SI,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,EXTRANJERA,True,5.0,2020.0,5,2020
1915,2020-09-12,15843c,2020-05-20,2020-05-20,NaT,68,1,Estados Unidos de AmÃ©rica,99,MÉXICO,HUIXQUILUCAN,MÉXICO,NO ESPECIFICADO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO APLICA,FUERA DE USMER,PRIVADA,HOMBRE,AMBULATORIO,EXTRANJERA,False,NaN,NaN,5,2020


Filtrado múltiple

In [78]:
# usando query podemos hacer filtrados múltiples
data.query('nacionalidad == "MEXICANA" & entidad_res == "SONORA" ').head(4)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
446,2020-09-12,1136dd,2020-03-27,2020-03-26,NaT,52,1,MÃ©xico,99,SONORA,SAN LUIS RÍO COLORADO,SONORA,SONORA,NO APLICA,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,3,2020
458,2020-09-12,0182cf,2020-05-20,2020-05-17,NaT,57,1,MÃ©xico,99,SONORA,NACOZARI DE GARCÍA,SONORA,SONORA,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,5,2020
469,2020-09-12,1b2101,2020-05-11,2020-05-08,NaT,27,1,MÃ©xico,99,SONORA,CAJEME,SONORA,SINALOA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False,NaN,NaN,5,2020
511,2020-09-12,0d9efa,2020-05-18,2020-05-12,NaT,46,1,MÃ©xico,99,SONORA,CAJEME,SONORA,SONORA,NO APLICA,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False,NaN,NaN,5,2020


In [79]:
# usando el filtrado tradicional debemos encerrar en paréntesis cada condición que queremos que cumpla el filtro
data[(data['nacionalidad'] == "MEXICANA") & (data.entidad_res == "JALISCO")].head(3)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
5,2020-09-12,16267b,2020-06-29,2020-06-27,NaT,24,1,MÃ©xico,99,JALISCO,GUADALAJARA,JALISCO,JALISCO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,CRUZ ROJA,HOMBRE,AMBULATORIO,MEXICANA,False,NaN,NaN,6,2020
25,2020-09-12,0f11e7,2020-05-19,2020-05-18,NaT,35,1,MÃ©xico,99,JALISCO,PUERTO VALLARTA,JALISCO,JALISCO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False,NaN,NaN,5,2020
106,2020-09-12,0c0a4f,2020-05-04,2020-04-29,NaT,45,1,MÃ©xico,99,JALISCO,GUADALAJARA,JALISCO,JALISCO,NO,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,False,NaN,NaN,5,2020


Resúmenes

In [27]:
# una columna una métrica
data.groupby('sexo').edad.mean()

sexo
HOMBRE    42.667259
MUJER     41.639353
Name: edad, dtype: float64

In [81]:
# una columna varias métricas
data.groupby('sexo').agg({'edad':['min','mean','max']})

edad                
        min       mean  max
sexo                       
HOMBRE    0  42.667259  120
MUJER     0  41.639353  120

In [82]:
# varias columnas una métrica 
data.groupby('sexo').agg({'edad': 'mean', 'mes_ingreso': 'mean', 'mes_def': 'mean'})

,edad,mes_ingreso,mes_def
sexo,,,
HOMBRE,42.667259,6.730779,6.497591
MUJER,41.639353,6.807770,6.572409


In [84]:
# varias columnas, varias métricas
data.groupby('sexo').agg({'edad': ['min', 'mean', 'max'], 'mes_ingreso': ['min', 'mean', 'max'], 'mes_def':['min', 'mean', 'max']})

edad                 mes_ingreso               mes_def               
        min       mean  max         min      mean max     min      mean  max
sexo                                                                        
HOMBRE    0  42.667259  120           1  6.730779   9     1.0  6.497591  9.0
MUJER     0  41.639353  120           1  6.807770   9     1.0  6.572409  9.0

Descripción por tipo

In [86]:
# Filtrar por tipos generales (categóricas y numéricas)
numerical = data.select_dtypes(include='number')
categorical = data.select_dtypes(include=['object', 'bool', 'category', 'datetime64[ns]'])


In [87]:
numerical.head()

,edad,resultado,mes_def,año_def,mes_ingreso,año_ingreso
0,52,1,NaN,NaN,7,2020
1,57,1,5.0,2020.0,4,2020
2,44,1,NaN,NaN,4,2020
3,28,1,NaN,NaN,4,2020
4,53,1,NaN,NaN,4,2020


In [88]:
categorical.head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion
0,2020-09-12,114cd3,2020-07-06,2020-06-29,NaT,MÃ©xico,99,TABASCO,CENTRO,TABASCO,TABASCO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,MUJER,AMBULATORIO,MEXICANA,False
1,2020-09-12,0bafd8,2020-04-29,2020-04-28,2020-05-09,MÃ©xico,99,MÉXICO,TOLUCA,MÉXICO,MÉXICO,SI,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,True
2,2020-09-12,18747a,2020-04-14,2020-04-10,NaT,MÃ©xico,99,SINALOA,MAZATLÁN,SINALOA,SINALOA,NO APLICA,NO,NO,NO,SI,NO,NO,NO,SI,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,False
3,2020-09-12,174174,2020-04-18,2020-04-11,NaT,MÃ©xico,99,CIUDAD DE MÉXICO,IZTAPALAPA,CIUDAD DE MÉXICO,GUERRERO,NO APLICA,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,False
4,2020-09-12,0eae86,2020-04-21,2020-04-21,NaT,MÃ©xico,99,VERACRUZ DE IGNACIO DE LA LLAVE,ACTOPAN,VERACRUZ DE IGNACIO DE LA LLAVE,MÉXICO,SI,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,SI,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,False


In [90]:
# para conocer todos los tipos de datos
?data.select_dtypes

Categóricas

In [91]:
# Cardinalidad Valores únicos por columna
for col in categorical.columns:
  print('Valores únicos en "{}": {}'.format(col,categorical [col].nunique()))

Valores únicos en "fecha_actualizacion": 1
Valores únicos en "id_registro": 1510311
Valores únicos en "fecha_ingreso": 256
Valores únicos en "fecha_sintomas": 256
Valores únicos en "fecha_def": 201
Valores únicos en "pais_nacionalidad": 119
Valores únicos en "pais_origen": 77
Valores únicos en "entidad_res": 32
Valores únicos en "municipio_res": 2164
Valores únicos en "entidad_um": 32
Valores únicos en "entidad_nac": 33
Valores únicos en "intubado": 4
Valores únicos en "neumonia": 3
Valores únicos en "embarazo": 4
Valores únicos en "habla_lengua_indig": 3
Valores únicos en "diabetes": 3
Valores únicos en "epoc": 3
Valores únicos en "asma": 3
Valores únicos en "inmusupr": 3
Valores únicos en "hipertension": 3
Valores únicos en "otra_com": 3
Valores únicos en "cardiovascular": 3
Valores únicos en "obesidad": 3
Valores únicos en "renal_cronica": 3
Valores únicos en "tabaquismo": 3
Valores únicos en "otro_caso": 3
Valores únicos en "migrante": 3
Valores únicos en "uci": 4
Valores únicos en

In [92]:
# unicidad - proporción de valores únicos por columna
total = data.shape[0]
for col in categorical.columns:
  print('Porcentaje de valores únicos en "{}": {}%'.format(col,round(categorical[col].nunique()/total*100,2)))

Porcentaje de valores únicos en "fecha_actualizacion": 0.0%
Porcentaje de valores únicos en "id_registro": 100.0%
Porcentaje de valores únicos en "fecha_ingreso": 0.02%
Porcentaje de valores únicos en "fecha_sintomas": 0.02%
Porcentaje de valores únicos en "fecha_def": 0.01%
Porcentaje de valores únicos en "pais_nacionalidad": 0.01%
Porcentaje de valores únicos en "pais_origen": 0.01%
Porcentaje de valores únicos en "entidad_res": 0.0%
Porcentaje de valores únicos en "municipio_res": 0.14%
Porcentaje de valores únicos en "entidad_um": 0.0%
Porcentaje de valores únicos en "entidad_nac": 0.0%
Porcentaje de valores únicos en "intubado": 0.0%
Porcentaje de valores únicos en "neumonia": 0.0%
Porcentaje de valores únicos en "embarazo": 0.0%
Porcentaje de valores únicos en "habla_lengua_indig": 0.0%
Porcentaje de valores únicos en "diabetes": 0.0%
Porcentaje de valores únicos en "epoc": 0.0%
Porcentaje de valores únicos en "asma": 0.0%
Porcentaje de valores únicos en "inmusupr": 0.0%
Porcenta

In [93]:
#Moda- valor que más se repite en cada columna excepto los indicadores únicos
for col in categorical.drop(columns= 'id_registro').columns:
  print('Valor más popular en "{}": {}'.format(col, statistics.mode(categorical[col])))

Valor más popular en "fecha_actualizacion": 2020-09-12 00:00:00
Valor más popular en "fecha_ingreso": 2020-07-20 00:00:00
Valor más popular en "fecha_sintomas": 2020-07-20 00:00:00
Valor más popular en "fecha_def": NaT
Valor más popular en "pais_nacionalidad": MÃ©xico
Valor más popular en "pais_origen": 99
Valor más popular en "entidad_res": CIUDAD DE MÉXICO
Valor más popular en "municipio_res": IZTAPALAPA
Valor más popular en "entidad_um": CIUDAD DE MÉXICO
Valor más popular en "entidad_nac": CIUDAD DE MÉXICO
Valor más popular en "intubado": NO APLICA
Valor más popular en "neumonia": NO 
Valor más popular en "embarazo": NO 
Valor más popular en "habla_lengua_indig": NO 
Valor más popular en "diabetes": NO 
Valor más popular en "epoc": NO 
Valor más popular en "asma": NO 
Valor más popular en "inmusupr": NO 
Valor más popular en "hipertension": NO 
Valor más popular en "otra_com": NO 
Valor más popular en "cardiovascular": NO 
Valor más popular en "obesidad": NO 
Valor más popular en "r

Numéricas

In [3]:
numerical.describe()

NameError: ignored

Análisis exploratorio 

In [95]:
#¿Cuántas pruebas por origen se realizan?
data.groupby('origen').id_registro.count()

origen
FUERA DE USMER    1010999
USMER              499312
Name: id_registro, dtype: int64

In [96]:
data.origen.value_counts()

FUERA DE USMER    1010999
USMER              499312
Name: origen, dtype: int64

In [97]:
#¿Cuántas pruebas por sector se realizan?
data.sector.value_counts()

SSA                942672
IMSS               386225
ISSSTE              57072
PRIVADA             55614
ESTATAL             27163
PEMEX               12919
IMSS-BIENESTAR      10158
SEMAR                7796
SEDENA               7371
MUNICIPAL            1201
UNIVERSITARIO        1046
DIF                   843
CRUZ ROJA             223
NO ESPECIFICADO         8
Name: sector, dtype: int64

In [98]:
#¿Cuántos pacientes por sexo tenemos?
data.sexo.value_counts()

MUJER     768635
HOMBRE    741676
Name: sexo, dtype: int64

In [99]:
#¿Cuál es la proporción de pacientes por sexo?
data.sexo.value_counts(normalize=True)

MUJER     0.508925
HOMBRE    0.491075
Name: sexo, dtype: float64

In [100]:
#¿Cuál es la proporción de resultados de las pruebas?
data.resultado.value_counts(normalize=True)

2    0.502670
1    0.439627
3    0.057703
Name: resultado, dtype: float64

In [105]:
#¿Cuántas pruebas positivas tenemos en el set?
data[data.resultado == 'Resultado pendiente'].id_registro.count()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


0

In [106]:
#¿Cuántos casos positivos tenemos por mes?
data[data.resultado == 'Resultado pendiente'].mes_ingreso.value_counts().sort_index()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Series([], Name: mes_ingreso, dtype: int64)

In [28]:
# ¿Cuál es el resultado en las pruebas por sexo?
data.groupby(['sexo', 'resultado'])['id_registro'].count()

sexo    resultado
HOMBRE  1            346464
        2            352619
        3             42593
MUJER   1            317509
        2            406569
        3             44557
Name: id_registro, dtype: int64

In [30]:
data.groupby('sexo').resultado.value_counts()

sexo    resultado
HOMBRE  2            352619
        1            346464
        3             42593
MUJER   2            406569
        1            317509
        3             44557
Name: resultado, dtype: int64

In [32]:
# ¿Qué proporción de resultados de pruebas tenemos por sexo?
data.groupby('sexo').resultado.value_counts(normalize=True)

sexo    resultado
HOMBRE  2            0.475435
        1            0.467137
        3            0.057428
MUJER   2            0.528949
        1            0.413082
        3            0.057969
Name: resultado, dtype: float64

In [33]:
#¿Cuál es la edad de los pacientes por sexo?
data.groupby('sexo').edad.describe()

,count,mean,std,min,25%,50%,75%,max
sexo,,,,,,,,
HOMBRE,741676.0,42.667259,17.126286,0.0,30.0,41.0,54.0,120.0
MUJER,768635.0,41.639353,16.479581,0.0,30.0,40.0,52.0,120.0


In [36]:
def q25(x):
  return x.quantile(0.25)

  def q75(x):
    return x.quantile(0.75)

 data.groupby('sexo').agg({'edad': ['min', q25, 'median', q75, 'max']})   


IndentationError: ignored

In [37]:
# ¿Cuál es la edad de los pacientes dependiendo si son entubados o no?
data.groupby('intubado').edad.describe()


,count,mean,std,min,25%,50%,75%,max
intubado,,,,,,,,
NO,221572.0,53.304786,19.684371,0.0,42.0,55.0,67.0,118.0
NO APLICA,1248894.0,39.686951,15.079782,0.0,29.0,38.0,49.0,120.0
NO ESPECIFICADO,194.0,54.443299,16.817502,0.0,45.0,57.0,65.0,92.0
SI,39651.0,57.111826,17.914277,0.0,48.0,59.0,69.0,108.0


In [38]:
#¿Cuál es la edad máxima y mínima de los pacientes fallecidos?
edad_def_max = data[data.defuncion].edad.max()
data[data.defuncion & (data.edad == edad_def_max)]

,fecha_actualizacion,id_registro,entidad_res,municipio_res,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
581442,2020-09-12,1b7b20,16,6,2020-05-21,2020-05-20,2020-05-26,109,2,MÃ©xico,99,MICHOACÁN DE OCAMPO,MICHOACÁN DE OCAMPO,NO,SI,NO APLICA,NO,NO,NO,NO,SI,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ISSSTE,HOMBRE,HOSPITALIZADO,MEXICANA,True,5.0,2020.0,5,2020


In [41]:
#¿Cuál es la edad  mínima de los pacientes fallecidos?
edad_def_min = data[data.defuncion].edad.min()
data[data.defuncion & (data.edad == edad_def_min)]

,fecha_actualizacion,id_registro,entidad_res,municipio_res,fecha_ingreso,fecha_sintomas,fecha_def,edad,resultado,pais_nacionalidad,pais_origen,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
11277,2020-09-12,18feee,2,4,2020-06-23,2020-06-23,2020-07-06,0,1,MÃ©xico,99,BAJA CALIFORNIA,BAJA CALIFORNIA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,True,7.0,2020.0,6,2020
11565,2020-09-12,083c34,15,31,2020-05-25,2020-05-25,2020-06-01,0,1,MÃ©xico,99,MÉXICO,MÉXICO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,SI,USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,True,6.0,2020.0,5,2020
26116,2020-09-12,02f8e2,2,4,2020-05-08,2020-05-01,2020-05-10,0,1,MÃ©xico,99,BAJA CALIFORNIA,BAJA CALIFORNIA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,True,5.0,2020.0,5,2020
50494,2020-09-12,014b5d,2,4,2020-05-18,2020-05-17,2020-05-18,0,1,MÃ©xico,99,BAJA CALIFORNIA,BAJA CALIFORNIA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ISSSTE,HOMBRE,HOSPITALIZADO,MEXICANA,True,5.0,2020.0,5,2020
60925,2020-09-12,053398,1,1,2020-06-11,2020-06-11,2020-06-17,0,1,MÃ©xico,99,AGUASCALIENTES,AGUASCALIENTES,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,True,6.0,2020.0,6,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439056,2020-09-12,0860f4,21,156,2020-06-30,2020-06-30,2020-07-13,0,3,MÃ©xico,99,PUEBLA,PUEBLA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,True,7.0,2020.0,6,2020
1462455,2020-09-12,1e94e4,24,28,2020-08-14,2020-08-06,2020-08-20,0,3,MÃ©xico,99,SAN LUIS POTOSÍ,SAN LUIS POTOSÍ,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,SI,FUERA DE USMER,PRIVADA,HOMBRE,HOSPITALIZADO,MEXICANA,True,8.0,2020.0,8,2020
1473848,2020-09-12,00f39c,14,121,2020-07-06,2020-07-03,2020-07-08,0,3,MÃ©xico,99,JALISCO,JALISCO,NO,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,SI,USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,True,7.0,2020.0,7,2020
1487374,2020-09-12,07cf0b,8,37,2020-08-26,2020-08-25,2020-08-27,0,3,MÃ©xico,99,CHIHUAHUA,COAHUILA DE ZARAGOZA,NO,SI,NO,NO,NO,NO,NO,NO,NO,SE IGNORA,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,USMER,SSA,MUJER,HOSPITALIZADO,MEXICANA,True,8.0,2020.0,8,2020


In [40]:
#¿ Cuál es la edad de los pacientes que fallecen?
data.groupby('defuncion').edad.describe()

,count,mean,std,min,25%,50%,75%,max
defuncion,,,,,,,,
False,1420116.0,40.888838,16.104672,0.0,29.0,39.0,51.0,120.0
True,90195.0,61.908676,15.218544,0.0,53.0,63.0,73.0,109.0


In [44]:
#¿Cuántas defunciones tenemos por mes?
data[data.defuncion].mes_def.value_counts().sort_index()

1.0        6
2.0       12
3.0      232
4.0     5294
5.0    16091
6.0    21461
7.0    23855
8.0    18791
9.0     4453
Name: mes_def, dtype: int64

In [45]:
#¿Cuál es la proporción de diabéticos para las defunciones?
data.groupby('diabetes').defuncion.value_counts(normalize=True)

diabetes   defuncion
NO         False        0.958145
           True         0.041855
SE IGNORA  False        0.873868
           True         0.126132
SI         False        0.813339
           True         0.186661
Name: defuncion, dtype: float64

In [47]:
#¿Cuál es la proporcion de fumadores para las defunciones?
data.groupby('tabaquismo').defuncion.value_counts(normalize=True)

tabaquismo  defuncion
NO          False        0.940671
            True         0.059329
SE IGNORA   False        0.869078
            True         0.130922
SI          False        0.938380
            True         0.061620
Name: defuncion, dtype: float64